In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.express as px
import seaborn as sns
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

In [20]:
global_data=pd.read_csv('cleaned_for_global_vis.csv')
global_data

,Unnamed: 0,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,...,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,mean_monthly_earnings,mean_yearly_earnings,lat,lon
0,0,378,Linkin Park,20400000,1.339700e+10,Music,linkinpark,0,Afghanistan,AFG,...,Nov,28.0,9.7,38041754.0,11.12,9797273.0,0.000,0.00,34.790665,64.092615
1,1,664,Musas,15100000,2.400038e+09,Howto & Style,Jordi Sala,159,Andorra,AND,...,Mar,12.0,NaN,NaN,NaN,NaN,0.180,2.13,42.531663,1.481332
2,2,77,shfa2 - ï¿½ï¿½,39700000,2.388482e+10,People & Blogs,shfa2 - ï¿½ï¿½,1596,United Arab Emirates,ARE,...,Nov,6.0,36.8,9770529.0,2.35,8479744.0,526400.000,6321600.00,23.644294,54.831324
3,3,256,shfa show India,24200000,1.572416e+10,Other,shfa show India,469,United Arab Emirates,ARE,...,Apr,1.0,36.8,9770529.0,2.35,8479744.0,321300.000,3876800.00,23.514234,54.866108
4,4,333,ZAMZAM ELECTRONICS TRADING,21700000,8.507819e+09,Other,ZAMZAM ELECTRONICS TRADING,15126,United Arab Emirates,ARE,...,Dec,19.0,36.8,9770529.0,2.35,8479744.0,3399800.000,40750000.00,23.859475,54.107903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,864,152,YOLO AVENTURAS,31200000,9.673649e+09,People & Blogs,YOLO AVENTURAS,65,Venezuela,VEN,...,Dec,16.0,79.3,28515829.0,8.80,25162368.0,7.450,91.50,4.625386,-65.016287
865,865,490,Like Nastya VNM,17900000,9.867516e+09,Entertainment,Like Nastya VNM,602,Vietnam,VNM,...,Nov,22.0,28.5,96462106.0,2.01,35332140.0,179500.000,2176700.00,22.743096,105.860458
866,866,586,POPS Kids,16200000,9.763593e+09,Music,POPS Kids,3943,Vietnam,VNM,...,Jan,14.0,28.5,96462106.0,2.01,35332140.0,532100.000,6535100.00,18.770431,105.496476
867,867,828,FAPTV,13700000,5.457204e+09,Comedy,FAPTV,863,Vietnam,VNM,...,Feb,26.0,28.5,96462106.0,2.01,35332140.0,74650.000,902700.00,14.531644,107.595761


In [21]:
global_data.isna().sum()

Unnamed: 0                                   0
rank                                         0
Youtuber                                     0
subscribers                                  0
video views                                  0
category                                     0
Title                                        0
uploads                                      0
Country                                      0
Abbreviation                                 0
channel_type                                 0
video_views_rank                             1
country_rank                                 1
channel_type_rank                            9
video_views_for_the_last_30_days            22
lowest_monthly_earnings                      0
highest_monthly_earnings                     0
lowest_yearly_earnings                       0
highest_yearly_earnings                      0
subscribers_for_last_30_days               276
created_year                                 0
created_month

In [22]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Assuming global_data includes necessary country-aggregated data
# Example columns: 'Country', 'Latitude', 'Longitude', 'subscribers', 'video views', 'Unemployment_rate', 'Population'

app.layout = html.Div([
    html.H1("Global YouTube Data Analysis"),
    dcc.Dropdown(
        id='youtube-metric-dropdown',
        options=[
            {'label': 'Subscribers', 'value': 'subscribers'},
            {'label': 'Video Views', 'value': 'video views'},
            {'label': 'New Video Views', 'value': 'video_views_for_the_last_30_days'},
            {'label': 'New Subscribers', 'value': 'subscribers_for_last_30_days'},
            {'label': 'Uploads', 'value': 'uploads'}
        ],
        value='subscribers',  # Default value
        clearable=False
    ),
    dcc.Graph(id='global-youtube-data-visualization')
])

@app.callback(
    Output('global-youtube-data-visualization', 'figure'),
    Input('youtube-metric-dropdown', 'value')
)
def update_map(selected_metric):
    # print(selected_metric)
    if selected_metric=='subscribers_for_last_30_days':
        df=global_data.dropna(subset=['subscribers_for_last_30_days']).copy()
    elif selected_metric=='video_views_for_the_last_30_days':
        df=global_data.dropna(subset=['video_views_for_the_last_30_days']).copy()
    else:
        df=global_data.copy()
    fig = px.scatter_geo(df,
        lat='lat',
        lon='lon',
        color='category',
        size=selected_metric,
        hover_name='Youtuber',
        hover_data=['Title','Country','highest_yearly_earnings','mean_yearly_earnings','created_year','created_month'],
        projection="natural earth",
        title=f"Global Distribution of Youtube Channels based on {selected_metric.capitalize()}"
    )
    fig.update_layout(
        margin={"r":0, "t":0, "l":0, "b":0},
        height=600,  # Set the height of the plot
        width=1200  # Set the width of the plot
    )
    fig.update_geos(
        visible=True,
        showcountries=True,
        countrycolor="Lightgrey",
        showland=True,
        landcolor='whitesmoke',
        lakecolor='LightBlue',
        showocean=True,
        oceancolor='Azure'
    )
    # Use a light color scale
    # fig.update_traces(marker_line_width=0, colorscale="Mint")
    # # Remove map background and add labels
    # # In the app.callback function where the figure is generated
    # fig.update_geos(
    #     visible=True,  # Ensure the entire geo axes are visible
    #     showcountries=True,  # Show country borders
    #     countrycolor="Lightgrey",  # Light grey color for country borders for subtle outline
    #     showcoastlines=False,  # Turn off coastlines if desired, or set to True if needed
    #     showland=True,  # Ensure land is shown
    #     landcolor='whitesmoke'  # Very light color for land
    # )

    fig.update_layout(
        geo=dict(
            bgcolor= 'rgba(255,255,255,0)',  # Transparent background
            lakecolor='LightBlue',  # Light blue for water bodies
            showocean=True,  # Show ocean
            oceancolor='Azure'  # Very light blue for ocean
        )
    )

    # If you want to add country names, you need to manually add them as annotations, which can be complex.
    # Here's a simplified approach to add annotations. In practice, you might need to adjust positions or skip if there are too many countries.
    # annotations = [
    #     dict(
    #         x=country_info['Longitude'],  # Assuming there's a Longitude for label placement
    #         y=country_info['Latitude'],   # Assuming there's a Latitude for label placement
    #         xref='x',
    #         yref='y',
    #         text=country_info['Country'],  # Country name
    #         showarrow=False,
    #         font=dict(
    #             family="Courier New, monospace",
    #             size=12,
    #             color="#ffffff"
    #         )
    #     ) for index, country_info in global_data.iterrows()
    # ]
    fig.update_layout()
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port="9091")


In [10]:
global_data

,Unnamed: 0,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,...,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,mean_monthly_earnings,mean_yearly_earnings,lat,lon
0,0,378,Linkin Park,20400000,1.339700e+10,Music,linkinpark,0,Afghanistan,AFG,...,Nov,28.0,9.7,38041754.0,11.12,9797273.0,0.000,0.00,34.790665,64.092615
1,1,664,Musas,15100000,2.400038e+09,Howto & Style,Jordi Sala,159,Andorra,AND,...,Mar,12.0,NaN,NaN,NaN,NaN,0.180,2.13,42.531663,1.481332
2,2,77,shfa2 - ï¿½ï¿½,39700000,2.388482e+10,People & Blogs,shfa2 - ï¿½ï¿½,1596,United Arab Emirates,ARE,...,Nov,6.0,36.8,9770529.0,2.35,8479744.0,526400.000,6321600.00,23.644294,54.831324
3,3,256,shfa show India,24200000,1.572416e+10,Other,shfa show India,469,United Arab Emirates,ARE,...,Apr,1.0,36.8,9770529.0,2.35,8479744.0,321300.000,3876800.00,23.514234,54.866108
4,4,333,ZAMZAM ELECTRONICS TRADING,21700000,8.507819e+09,Other,ZAMZAM ELECTRONICS TRADING,15126,United Arab Emirates,ARE,...,Dec,19.0,36.8,9770529.0,2.35,8479744.0,3399800.000,40750000.00,23.859475,54.107903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,864,152,YOLO AVENTURAS,31200000,9.673649e+09,People & Blogs,YOLO AVENTURAS,65,Venezuela,VEN,...,Dec,16.0,79.3,28515829.0,8.80,25162368.0,7.450,91.50,4.625386,-65.016287
865,865,490,Like Nastya VNM,17900000,9.867516e+09,Entertainment,Like Nastya VNM,602,Vietnam,VNM,...,Nov,22.0,28.5,96462106.0,2.01,35332140.0,179500.000,2176700.00,22.743096,105.860458
866,866,586,POPS Kids,16200000,9.763593e+09,Music,POPS Kids,3943,Vietnam,VNM,...,Jan,14.0,28.5,96462106.0,2.01,35332140.0,532100.000,6535100.00,18.770431,105.496476
867,867,828,FAPTV,13700000,5.457204e+09,Comedy,FAPTV,863,Vietnam,VNM,...,Feb,26.0,28.5,96462106.0,2.01,35332140.0,74650.000,902700.00,14.531644,107.595761
